# Data loaders

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from qadence.ml_tools import DictDataLoader, to_dataloader


def dataloader(data_size: int = 25, batch_size: int = 5, infinite: bool = False) -> DataLoader:
    x = torch.linspace(0, 1, data_size).reshape(-1, 1)
    y = torch.sin(x)
    return to_dataloader(x, y, batch_size=batch_size, infinite=infinite)


def dictdataloader(data_size: int = 25, batch_size: int = 5) -> DictDataLoader:
    dls = {}
    for k in ["y1", "y2"]:
        x = torch.rand(data_size, 1)
        y = torch.sin(x)
        dls[k] = to_dataloader(x, y, batch_size=batch_size, infinite=True)
    return DictDataLoader(dls)


# iterate over standard DataLoader
for (x, y) in dataloader(data_size=6, batch_size=2):
    print(f"Standard {x = }")

# construct an infinite dataset which will keep sampling indefinitely
n_epochs = 5
dl = iter(dataloader(data_size=6, batch_size=2, infinite=True))
for _ in range(n_epochs):
    (x, y) = next(dl)
    print(f"Infinite {x = }")

# iterate over DictDataLoader
ddl = dictdataloader()
data = next(iter(ddl))
print(f"{data = }")

Standard x = tensor([[0.0000],
        [0.2000]])
Standard x = tensor([[0.4000],
        [0.6000]])
Standard x = tensor([[0.8000],
        [1.0000]])
Infinite x = tensor([[0.],
        [1.]])
Infinite x = tensor([[0.8000],
        [0.4000]])
Infinite x = tensor([[0.6000],
        [0.2000]])
Infinite x = tensor([[0.],
        [1.]])
Infinite x = tensor([[0.8000],
        [0.4000]])
data = {'y1': [tensor([[0.0153],
        [0.3496],
        [0.8334],
        [0.9558],
        [0.1501]]), tensor([[0.0153],
        [0.3425],
        [0.7402],
        [0.8168],
        [0.1495]])], 'y2': [tensor([[0.1908],
        [0.4887],
        [0.9825],
        [0.9339],
        [0.3075]]), tensor([[0.1896],
        [0.4695],
        [0.8319],
        [0.8039],
        [0.3027]])]}


# Train configs

| Attribute              | Type                          | Default      | Description                                      |
|------------------------|-----------------------------|-------------|--------------------------------------------------|
| max_iter              | int                          | 10000       | Total number of training epochs.                |
| batch_size           | int                          | 1           | Batch size for training.                        |
| print_every          | int                          | 0           | Frequency of console output. Set to 0 to disable. |
| write_every          | int                          | 0           | Frequency of logging metrics. Set to 0 to disable. |
| plot_every           | int                          | 0           | Frequency of plotting metrics. Set to 0 to disable. |
| checkpoint_every     | int                          | 0           | Frequency of saving checkpoints. Set to 0 to disable. |
| val_every           | int                          | 0           | Frequency of validation checks. Set to 0 to disable. |
| val_epsilon         | float                        | 1e-5        | Threshold for validation improvement.           |
| validation_criterion | Callable                     | None        | Function for validating metric improvement.     |
| trainstop_criterion | Callable                     | None        | Function to stop training early.                |
| callbacks           | list[Callback]               | []          | List of custom callbacks.                       |
| root_folder        | Path                          | "./qml_logs" | Root directory for saving logs and checkpoints. |
| log_folder         | Path                          | "./qml_logs" | Logging directory for saving logs and checkpoints. |
| log_model          | bool                          | False       | Enables model logging.                          |
| verbose            | bool                          | True        | Enables detailed logging.                       |
| tracking_tool      | ExperimentTrackingTool       | TENSORBOARD | Tool for tracking training metrics.             |
| plotting_functions | tuple                        | ()          | Functions for plotting metrics.                 |


In [2]:
from qadence.ml_tools import OptimizeResult, TrainConfig
from qadence.ml_tools.callbacks import Callback

batch_size = 5
n_epochs = 100

print_parameters = lambda opt_res: print(opt_res.model.parameters())
condition_print = lambda opt_res: opt_res.loss < 1.0e-03
modify_extra_opt_res = {"n_epochs": n_epochs}
custom_callback = Callback(on="train_end", callback=print_parameters, callback_condition=condition_print,
                           modify_optimize_result=modify_extra_opt_res, called_every=10, )

config = TrainConfig(
    root_folder="some_path/",
    max_iter=n_epochs,
    checkpoint_every=100,
    write_every=100,
    batch_size=batch_size,
    callbacks=[custom_callback]
)


# Callbacks

In [4]:
from qadence.ml_tools.callbacks import Callback


def validation_criterion(val_loss: float, best_val_loss: float, val_epsilon: float) -> bool:
    return val_loss < (best_val_loss - val_epsilon)


def callback_fn(trainer, config, writer):
    if trainer.opt_res.loss < 0.001:
        print("Custom Callback: Loss threshold reached!")


custom_callback = Callback(on="train_epoch_end", called_every=10, callback=callback_fn)

config = TrainConfig(callbacks=[custom_callback])